In [ ]:
from astropy.coordinates import SkyCoord

NAME = "CRTS J033349.8-282244"
coord = SkyCoord.from_name(NAME)
RA, DEC = coord.ra.deg, coord.dec.deg

In [ ]:
import lsdb

cat = lsdb.open_catalog(
    "/sdf/data/rubin/user/kostya/hats/dia_object_lc_15k",
    search_filter=lsdb.ConeSearch(RA, DEC, radius_arcsec=0.2)
)
df = cat.compute()
assert len(df) == 1, f"{len(df) = }"
row = df.iloc[0]
row

In [ ]:
lc = row["diaObjectForcedSource"].query(
    "~psfFlux_flag"
    # " and ~psfDiffFlux_flag"
    " and ~pixelFlags_suspect"
    " and ~pixelFlags_saturated"
    " and ~pixelFlags_cr"
    " and ~pixelFlags_bad"
)
lc

In [ ]:
import matplotlib.pyplot as plt

COLORS = {'u': '#0c71ff', 'g': '#49be61', 'r': '#c61c00',
          'i': '#ffc200', 'z': '#f341a2', 'y': '#5d0000'}

for band in "ugrizy":
    data = lc.query(f"band == '{band}'")
    plt.errorbar(data["midpointMjdTai"], data["psfMag"], yerr=data["psfMagErr"], label=f"{band}", fmt='o', color=COLORS[band])

plt.gca().invert_yaxis()
plt.xlabel(f"MJD")
plt.ylabel("PSF mag")
plt.title(f"{NAME}\ndiaObjectID {row.diaObjectId}\nRA=${row.ra:.5f}$, Dec=${row.dec:.5f}$")
plt.legend(loc='upper left')
plt.savefig(f"{row.diaObjectId}.pdf", bbox_inches='tight')

In [ ]:
import numpy as np

BEFORE_PEAK_MJD = 60633

for band in 'ugrizy':
    idx_peak = lc.query(f"band == '{band}'")["psfMag"].idxmin()
    print(f"peak {band}-band: MJD = {lc.loc[idx_peak]["midpointMjdTai"]:.5f}, mag = {lc.loc[idx_peak]["psfMag"]:.3f} ± {lc.loc[idx_peak]["psfMagErr"]:.3f}")

    quiescent = lc.query("band == @band and midpointMjdTai < @BEFORE_PEAK_MJD")
    quiescent_mag = np.median(quiescent["psfMag"])
    print(f"quiescent {band} mag = {quiescent_mag:.3f}")